# S1 Routines

**Structural uncertainty due to fault timing: a multi-model case study from the Perth Basin**<br><br>Bardot, K., Lesueur, M., Siade, A. J., Lang, S. C. and McCallum, J. L. (2024)

In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import os
import flopy
import math
from scipy.interpolate import interp1d
import pandas as pd
import pickle

class Bore:  
    def __init__(self, easting, northing, ground):  
        self.easting = easting
        self.northing = northing
        self.ground = ground

class Model:
    
    def __init__(self, nlay, nrow, ncol):  
        self.nlay = nlay
        self.nrow = nrow
        self.ncol = ncol
        self.bot = np.ones((nlay,nrow, ncol))
        self.centres = np.ones((nlay,nrow, ncol))
        self.idomain = np.ones((nlay,nrow, ncol))
        self.top = np.ones((nrow, ncol))
        self.recharge = np.zeros((nrow, ncol))
        self.mask = np.ones((nlay,nrow, ncol))
        self.hk = np.ones((nlay,nrow, ncol))
        self.vk = np.ones((nlay,nrow, ncol))
        self.heads = np.zeros((nlay,nrow, ncol))
        self.angle1 = np.zeros((nlay,nrow, ncol))
        self.angle2 = np.zeros((nlay,nrow, ncol))
        self.angle3 = np.zeros((nlay,nrow, ncol))
        self.layers = np.zeros((nlay,nrow, ncol))
        
def grad_to_deg(grad):
    return(np.degrees(math.atan(grad)))

logfunc = lambda e: np.log10(e)

def extract_data(workspace, modelname):# EXTRACT DATA
    fpth = os.path.join(workspace, modelname +'.hds')
    hds = flopy.utils.binaryfile.HeadFile(fpth)  
    times = hds.get_times()
    head = hds.get_data(totim=times[-1])
    head_all = hds.get_alldata()
    print(times)
    
    fpth = os.path.join(workspace, modelname +'.cbc')
    cbc = flopy.utils.binaryfile.CellBudgetFile(fpth, precision=hds.precision)
    flowja = cbc.get_data(text='FLOW-JA-FACE')[-1][-1][-1]
    spd = cbc.get_data(text='DATA-SPDIS')[-1]
    
    nlay,nrow,ncol = head.shape[0], head.shape[1], head.shape[2]
    qx, qy, qz = np.zeros((nlay,nrow,ncol)), np.zeros((nlay,nrow,ncol)), np.zeros((nlay,nrow,ncol))
    qmag, qtheta = np.zeros((nlay,nrow,ncol)), np.zeros((nlay,nrow,ncol))
        
    for n in range(len(spd)):
        cell = spd[n][0]-1
        k,j,i = find_kji(cell,nlay,nrow,ncol)
        
        qx[k,j,i] = spd[n][3]
        qy[k,j,i] = spd[n][4]
        qz[k,j,i] = spd[n][5]
        qmag[k,j,i] = np.sqrt(spd[n][3]**2 + spd[n][4]**2 + spd[n][5]**2) 
        qtheta[k,j,i] = math.degrees(math.atan(spd[n][5]/abs(spd[n][3])))
    return(head, head_all, qmag, qtheta, spd, qx, qy, qz)

def x_to_col(x, delr):
    return(int(x/delr))
def z_to_lay(z, delz, zmax):
    return(int((zmax - z)/delz))

def growth_fault(x, strat):
        if x < AM72.x:
            a = 10 # y-stretch factor
            b = fault_x # x location of fault
            c = (a * (AM72.x - b)) ** 0.5 - getattr(AM72, strat) # Calculated based on strat at AM72
            y = (a*(x - b))**0.5 - c
            ygrad = 0.5 * (a * (x - b)**-1)**0.5
        if x >= AM72.x:
            m = 0.
            d = getattr(AM72, strat)-m*AM72.x
            y = m*x + d
            ygrad = m
        return(y, grad_to_deg(ygrad))

def faulted_bot_strat(xi, xmax, AM75strat, YMB1strat, AM72strat, strat):
    yi = np.zeros(len(xi))
    angle2 = np.zeros(len(xi))

    #Interpolate west of YMB1
    x1, x2, y1, y2 = AM75.x, YMB1.x, AM75strat, YMB1strat
    m_west = (y2 - y1) / (x2 - x1)
    c_west = y1 - m_west * x1

    #Interpolate between YMB1 and fault
    x1, x2, y1, y2 = AM75.x, YMB1.x, AM75strat, YMB1strat
    m_mid = 0.
    c_mid = y2

    #Interpolate east of fault until AM72  
    for i in range(len(xi)):
        x = xi[i]
        y1 = m_west * x + c_west # Linear West boundary to YMB1
        ymid = m_mid * x + c_mid # Flat YMB1 to Fault

        if x < YMB1.x: # West boundary to YMB1
            yi[i], angle2[i] = y1, grad_to_deg(m_west)

        if x >= YMB1.x and x <= fault_x: # YMB1 to fault
            yi[i], angle2[i] = ymid, grad_to_deg(m_mid)

        if x > fault_x: # Fault to east boundray
            yi[i], angle2[i] = growth_fault(x, strat)
        

    return(yi, angle2)

def x_to_col(x, delr): return(int(x/delr))
def z_to_lay(z, delz, zmax): return(int((zmax - z)/delz))

def find_kji(cellid,nlay,nrow,ncol): #cellid is zerobased  
    k = math.floor(cellid/(ncol*nrow)) 
    j = math.floor((cellid - k*ncol*nrow)/ncol) 
    i = math.floor(cellid - k*ncol*nrow - j*ncol) 
    return(k,j,i) # ZERO BASED!

print('routines run')

routines run
